In [ ]:
!pip install pyarrow
!pip install python-dotenv
!pip install snac peft soundfile

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import random
from ask_lisa import ask_lisa
from dotenv import load_dotenv
import json

In [ ]:
base_dir = "/training-1/asr_dataset_files/asr_bundestag"
train = base_dir + "/train_nodev"
validate = base_dir + "/train_dev"
test = base_dir + "/test"
wav_dir = "/training-1/asr/bundestag/dataset/wavs"
prompts = "../../system_prompts/"

load_dotenv()

# Data Exploration

In [ ]:
mapping = {}

with open(train + "/wav.scp", "r", encoding="utf-8") as f:
    for line in f:
        words = line.strip().split()
        mapping.update({words[0]: [words[1]]})

for i, (key, value) in enumerate(mapping.items()):
    if i >= 5:
        break
    print(key, ": ", value)

In [ ]:
with open(train + "/text", "r", encoding="utf-8") as f:
    for line in f:
        words = line.strip().split(maxsplit=1)
        mapping[words[0]].append(words[1])

for i, (key, value) in enumerate(mapping.items()):
    if i >= 2:
        break
    print(key, ": ", value)

In [ ]:
word_count = {}

with open(train + "/text", "r", encoding="utf-8") as f:
    for line in f:
        count = len(line.strip().split()) - 1
        if count in word_count:
            word_count[count] = word_count[count] + 1
        else:
            word_count[count] = 1

counts = sorted(word_count.keys())
frequencies = [word_count[c] for c in counts]

plt.bar(counts, frequencies)
plt.xlabel("Wörter pro Transkript")
plt.ylabel("Häufigkeit")
plt.show()

In [ ]:
filtered_mapping = {}
for key, value in mapping.items():
    count = len(value[1].split())
    if 70 < count:
        filtered_mapping.update({key: value})

mapping = filtered_mapping

for i, (key, value) in enumerate(mapping.items()):
    if i >= 2:
        break
    print(key, ": ", value)

print(len(mapping.items()))

In [ ]:
with open(prompts + "check_context_prompt.txt", "r", encoding="utf-8") as f:
    system_prompt_check_context = f.read()
    
with open(prompts + "generate_prompt.txt", "r", encoding="utf-8") as f:
    system_prompt = f.read()

with open(prompts + "verify_prompt.txt", "r", encoding="utf-8") as f:
    system_prompt_verify = f.read() + system_prompt

def check_context(context):
    res = ask_lisa(system_prompt_check_context, context)
    print(res)
    try:
        return json.loads(res).get("context") == "ok"
    except json.JSONDecodeError:
        print("Antwort war kein gültiges JSON:", res)
        return False

def generate_qa(context):
    qa_res = ask_lisa(system_prompt, context)

    try:
        qa_all = json.loads(qa_res)
    except json.JSONDecodeError:
        print("Antwort war kein gültiges JSON:", qa_res)
        return None

    verify = ask_lisa(
        system_prompt_verify,
        f"""
        Kontext:
        {context}
        Antwort:
        {qa_res}
        """
    )
    try:
        verified = json.loads(verify)
        print(verify)
        if verified.get("quality") == "ok":
            return qa_all
        else:
            print("Fragen korrigiert!")
            return verified.get("data")
    except json.JSONDecodeError:
        print("Antwort war kein gültiges JSON:", qa_res)
        return None

In [ ]:
# Ungeeigneter Kontext
context = "kein geschenk das ist verdient meine damen und herren durch dieses projekt ergeben sich man kann es kaum glauben großartige chancen merken sie sich das meine damen und herren kein anderes land kann so etwas vollbringen"
print(check_context(context))

# Ungeeigneter Kontext
context = "aufwendig und eigentlich nicht das was wir haben wollten da müsste noch mal ordentlich power reingesteckt werden um diese sache voranzutreiben ein anderer punkt der natürlich auch schwierig ist den sie sich vorstellen können ist der support wie es neu deutsch heißt also die unterstützung denn sie müssen sich das so vorstellen es haben auf einen satz sehr viele gesundheitsämter jetzt den anschluss gemacht und es ist immer sehr schwer in so einer wachstumsphase die"
print(check_context(context))

# Geeigneter Kontext
context = "es gibt zahlreiche nicht abklingende proteste es gibt initiativen es gibt self made projekte petitionen umfragen zeigen dass große teile der gesellschaft bereit sind jetzt wo wir hier reden haben wir vor dem bundestag eine begleitende kundgebung mit über zehn bewegungen und es gibt jetzt gerade redebeiträge und die zeigen ihnen auch wieder dass es nicht nur diese petition ist sondern dass es hunderttausende wenn nicht sogar millionen von menschen sind die tagtäglich immer wieder dafür kämpfen dass hier endlich was passiert im bereich der mobilitäts und verkehrswende zum schluss"
print(check_context(context))

# Geeigneter Kontext
context = "vielen dank für die gelegenheit frau ministerin ich freue mich auch sehr über ihre aussagen bezüglich der gehsteigbelästigung das ist ein sehr wichtiges thema wir sind gerade im kontakt mit unseren genossinnen und genossen der linksfraktion in hessen da ist das problem dass die dortige schwarz grüne regierung gerne auf den bund zeigt der dann wiederum auf die länder zeigt wie können sie denn garantieren dass diese regelung die sie jetzt erarbeiten dann auch wirklich umgesetzt wird also ist da etwas in"
print(check_context(context))

# Geeigneter Kontext
context = "alle mitgenommen und gezeigt wie wichtig die spracherziehung bereits in der kita ist es ist rundum evaluiert es ist von allen seiten als erfolgreich anerkannt deswegen ist jetzt der zeitpunkt dieses programm in die regelfinanzierung zu überführen das sieht auch das kita qualitätsgesetz vor eines der zentralen kriterien für die kitaqualität ist die sprachförderung deswegen setze ich mich dafür ein dass wir das gemeinsam hinkriegen vor allen dingen setze ich mich dafür ein dass wir mit den ländern auch zu einer guten übergangsregelung kommen sie wissen es"
print(check_context(context))

In [ ]:
generate_qa("es gibt zahlreiche nicht abklingende proteste es gibt initiativen es gibt self made projekte petitionen umfragen zeigen dass große teile der gesellschaft bereit sind jetzt wo wir hier reden haben wir vor dem bundestag eine begleitende kundgebung mit über zehn bewegungen und es gibt jetzt gerade redebeiträge und die zeigen ihnen auch wieder dass es nicht nur diese petition ist sondern dass es hunderttausende wenn nicht sogar millionen von menschen sind die tagtäglich immer wieder dafür kämpfen dass hier endlich was passiert im bereich der mobilitäts und verkehrswende zum schluss")

In [ ]:
# Zu verbessernde Fragen und Antworten
context = "es gibt zahlreiche nicht abklingende proteste es gibt initiativen es gibt self made projekte petitionen umfragen zeigen dass große teile der gesellschaft bereit sind jetzt wo wir hier reden haben wir vor dem bundestag eine begleitende kundgebung mit über zehn bewegungen und es gibt jetzt gerade redebeiträge und die zeigen ihnen auch wieder dass es nicht nur diese petition ist sondern dass es hunderttausende wenn nicht sogar millionen von menschen sind die tagtäglich immer wieder dafür kämpfen dass hier endlich was passiert im bereich der mobilitäts und verkehrswende zum schluss"
verify = ask_lisa(
system_prompt_verify,
f"""
Kontext:
{context}
Antwort:
{{
    "level_1": {{ "question": "Wer arbeitet?", "answer": "Bürger, Bewegungen" }},
    "level_2": {{ "question": "Was sagen die Politiker?", "answer": "Große Bereitschaft zur Verkehrswende" }},
    "level_3": {{ "question": "Warum sind die Proteste sinnvoll?", "answer": "Sie zeigen demokratische Partizipation" }}
}}
"""
)
print(verify)

# Anmerkung
**Die folgenden Zellen sind nicht mehr aktuell. Sie waren ein Prototyp. Die Funktionalität wurde in die eigentliche Datensatzgenerierung übernommen. Also bitte nicht wundern, wenn es nicht mehr richtig funktioniert**

In [ ]:
import torch
import os
import torch
from snac import SNAC
from datetime import datetime

from peft import PeftModel
import soundfile as sf
from transformers import AutoModelForCausalLM, AutoTokenizer

from huggingface_hub import login
login(token=os.getenv("HUGGINGFACE_TOKEN"))

model = AutoModelForCausalLM.from_pretrained(
    "SebastianBodza/Kartoffel_Orpheus-3B_german_synthetic-v0.1",
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(
    "SebastianBodza/Kartoffel_Orpheus-3B_german_synthetic-v0.1",
)

snac_model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz")
snac_model = snac_model.to("cuda")

chosen_voice = "Julian"

def process_single_prompt(prompt, chosen_voice):
    if chosen_voice == "in_prompt" or chosen_voice == "":
        full_prompt = prompt
    else:
        full_prompt = f"{chosen_voice}: {prompt}"
    start_token = torch.tensor([[128259]], dtype=torch.int64)
    end_tokens = torch.tensor([[128009, 128260]], dtype=torch.int64)

    input_ids = tokenizer(full_prompt, return_tensors="pt").input_ids
    modified_input_ids = torch.cat([start_token, input_ids, end_tokens], dim=1)

    input_ids = modified_input_ids.to("cuda")
    attention_mask = torch.ones_like(input_ids)

    generated_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=4000,
        do_sample=True,
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.1,
        num_return_sequences=1,
        eos_token_id=128258,
        use_cache=True,
    )

    token_to_find = 128257
    token_to_remove = 128258

    token_indices = (generated_ids == token_to_find).nonzero(as_tuple=True)

    if len(token_indices[1]) > 0:
        last_occurrence_idx = token_indices[1][-1].item()
        cropped_tensor = generated_ids[:, last_occurrence_idx + 1 :]
    else:
        cropped_tensor = generated_ids

    masked_row = cropped_tensor[0][cropped_tensor[0] != token_to_remove]
    row_length = masked_row.size(0)
    new_length = (row_length // 7) * 7
    trimmed_row = masked_row[:new_length]
    code_list = [t - 128266 for t in trimmed_row]

    return code_list


def redistribute_codes(code_list):
    layer_1 = []
    layer_2 = []
    layer_3 = []
    for i in range((len(code_list) + 1) // 7):
        layer_1.append(code_list[7 * i])
        layer_2.append(code_list[7 * i + 1] - 4096)
        layer_3.append(code_list[7 * i + 2] - (2 * 4096))
        layer_3.append(code_list[7 * i + 3] - (3 * 4096))
        layer_2.append(code_list[7 * i + 4] - (4 * 4096))
        layer_3.append(code_list[7 * i + 5] - (5 * 4096))
        layer_3.append(code_list[7 * i + 6] - (6 * 4096))

    codes = [
        torch.tensor(layer_1).unsqueeze(0),
        torch.tensor(layer_2).unsqueeze(0),
        torch.tensor(layer_3).unsqueeze(0),
    ]
    codes = [c.to("cuda") for c in codes]

    audio_hat = snac_model.decode(codes)
    return audio_hat

def to_speech(text):
    with torch.no_grad():
        code_list = process_single_prompt(text, chosen_voice)
        samples = redistribute_codes(code_list)

    audio_numpy = samples.detach().squeeze().to("cpu").numpy()
    
    # Audio in BytesIO Buffer schreiben statt in Datei
    buffer = BytesIO()
    sf.write(buffer, audio_numpy, 24000, format='WAV')
    buffer.seek(0)  # Zurück zum Anfang des Buffers
    
    return buffer.read()

In [ ]:
def to_speech(text):
    tts.tts_to_file(text=text, file_path="output.wav")
    with open("output.wav", "rb") as wav_file:
        return wav_file.read()

In [ ]:
keys = list(mapping.keys())

context_audio = []
context_text = []

q_1_audio = []
q_1_text = []
a_1_text = []

q_2_audio = []
q_2_text = []
a_2_text = []

q_3_audio = []
q_3_text = []
a_3_text = []

train_size = 2 #1200
val_size = 1 #150
test_size = 1 #150

In [ ]:
def generate_sample(key):
    context = mapping[key][1]
    context_text.append(context)
    with open(mapping[key][0], "rb") as wav_file:
        context_audio.append(wav_file.read())
    
    level_1, level_2, level_3 = generate_qa(mapping[key][1])

    q_1 = level_1.get("question")
    q_1_text.append(q_1)
    q_1_audio.append(to_speech(q_1))
    a_1_text.append(level_1.get("answer"))

    q_2 = level_2.get("question")
    q_2_text.append(q_2)
    q_2_audio.append(to_speech(q_2))
    a_2_text.append(level_2.get("answer"))

    q_3 = level_3.get("question")
    q_3_text.append(q_3)
    q_3_audio.append(to_speech(q_3))
    a_3_text.append(level_3.get("answer"))

In [ ]:
def to_df():
    return pd.DataFrame({
    "context_text": context_text,
    "context_audio": context_audio,
    "q_1_text": q_1_text,
    "q_1_audio": q_1_audio,
    "a_1_text": a_1_text,
    "q_2_text": q_2_text,
    "q_2_audio": q_2_audio,
    "a_2_text": a_2_text,
    "q_3_text": q_3_text,
    "q_3_audio": q_3_audio,
    "a_3_text": a_3_text
})

In [ ]:
def clear_arrays():
    context_audio.clear()
    context_text.clear()
    q_1_audio.clear()
    q_1_text.clear()
    a_1_text.clear()
    q_2_audio.clear()
    q_2_text.clear()
    a_2_text.clear()
    q_3_audio.clear()
    q_3_text.clear()
    a_3_text.clear()

In [ ]:
# Trainingsdatensatz
clear_arrays()

for i in range(train_size):
    key = random.choice(keys)
    generate_sample(key)
    keys.remove(key)

train_df = to_df()
train_df.to_parquet("train.parquet", engine="pyarrow")

In [ ]:
# Validierungsdatensatz
clear_arrays()

for i in range(val_size):
    key = random.choice(keys)
    generate_sample(key)
    keys.remove(key)

val_df = to_df()
val_df.to_parquet("validate.parquet", engine="pyarrow")

In [ ]:
clear_arrays()

# Testdatensatz
for i in range(test_size):
    key = random.choice(keys)
    generate_sample(key)
    keys.remove(key)

test_df = to_df()
test_df.to_parquet("test.parquet", engine="pyarrow")